In [1]:
# -*- coding: utf-8 -*-
"""
Final Working Lookalike Model
"""
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.compose import ColumnTransformer

# Load datasets
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')

# =============================================================================
# Step 1: Data Preparation & Feature Engineering
# =============================================================================

# Merge datasets with explicit column renaming
merged_data = pd.merge(
    pd.merge(
        transactions.rename(columns={'Price': 'TransactionPrice'}),
        products.rename(columns={'Price': 'ProductPrice'}),
        on='ProductID'
    ),
    customers,
    on='CustomerID'
)

# Create customer features with valid column references
customer_features = merged_data.groupby('CustomerID').agg({
    # Transaction behaviors
    'TransactionID': 'count',
    'TotalValue': ['sum', 'mean'],
    'Quantity': ['sum', 'mean'],
    'TransactionDate': lambda x: (pd.to_datetime(x).max() - pd.to_datetime(x).min()).days,
    
    # Product preferences
    'Category': lambda x: x.mode()[0] if not x.empty else None,
    'TransactionPrice': ['mean', 'max'],
    
    # Customer profile
    'Region': 'first',
    'SignupDate': lambda x: (pd.Timestamp.today() - pd.to_datetime(x).max()).days
}).reset_index()

# Flatten multi-index columns
customer_features.columns = [
    'CustomerID', 'TransactionCount', 'TotalSpend', 'AvgSpend',
    'TotalQuantity', 'AvgQuantity', 'TransactionSpanDays', 
    'FavoriteCategory', 'AvgProductPrice', 'MaxProductPrice',
    'Region', 'DaysSinceSignup'
]

# =============================================================================
# Step 2: Data Preprocessing
# =============================================================================

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), [
            'TransactionCount', 'TotalSpend', 'AvgSpend',
            'TotalQuantity', 'AvgQuantity', 'TransactionSpanDays',
            'AvgProductPrice', 'MaxProductPrice', 'DaysSinceSignup'
        ]),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Region', 'FavoriteCategory'])
    ],
    remainder='drop'
)

processed_features = preprocessor.fit_transform(customer_features)

# =============================================================================
# Step 3: Similarity Calculation
# =============================================================================

similarity_matrix = cosine_similarity(processed_features)
similarity_df = pd.DataFrame(
    similarity_matrix,
    index=customer_features['CustomerID'],
    columns=customer_features['CustomerID']
)

# =============================================================================
# Step 4: Generate Recommendations
# =============================================================================

def get_lookalikes(customer_id, n=3):
    """Return top N similar customers with scores"""
    similarities = similarity_df[customer_id].sort_values(ascending=False)
    return [(cust_id, round(score, 4)) 
            for cust_id, score in similarities.iloc[1:n+1].items()]

results = {}
target_customers = [f'C{str(i).zfill(4)}' for i in range(1, 21)]

for cust_id in target_customers:
    if cust_id in similarity_df.index:
        results[cust_id] = get_lookalikes(cust_id)
    else:
        results[cust_id] = [('Not Found', 0.0)] * 3

# =============================================================================
# Step 5: Save Results
# =============================================================================

output_df = pd.DataFrame({
    'CustomerID': results.keys(),
    'Lookalikes': results.values()
})

output_df.to_csv('Lookalike.csv', index=False)
print("Lookalike model executed successfully!")

Lookalike model executed successfully!


In [2]:
transactions.rename(columns={'Price': 'TransactionPrice'}),
products.rename(columns={'Price': 'ProductPrice'}),

(   ProductID              ProductName     Category  ProductPrice
 0       P001     ActiveWear Biography        Books        169.30
 1       P002    ActiveWear Smartwatch  Electronics        346.30
 2       P003  ComfortLiving Biography        Books         44.12
 3       P004            BookWorld Rug   Home Decor         95.69
 4       P005          TechPro T-Shirt     Clothing        429.31
 ..       ...                      ...          ...           ...
 95      P096     SoundWave Headphones  Electronics        307.47
 96      P097       BookWorld Cookbook        Books        319.34
 97      P098         SoundWave Laptop  Electronics        299.93
 98      P099   SoundWave Mystery Book        Books        354.29
 99      P100        HomeSense Sweater     Clothing        126.34
 
 [100 rows x 4 columns],)

In [3]:
print("Merged data columns:", merged_data.columns.tolist())

Merged data columns: ['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate', 'Quantity', 'TotalValue', 'TransactionPrice', 'ProductName', 'Category', 'ProductPrice', 'CustomerName', 'Region', 'SignupDate']
